In [ ]:
import csv
import os
import glob
from natsort import natsorted
import gc

In [ ]:
def translate(sequence): #TESTED
    """
    Translates a nucleotide sequence into an amino acid sequence.

    Arguments:
    sequence -- a string representing the nucleotide sequence

    Returns:
    A string representing the corresponding amino acid sequence
    """
    codon_table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    protein = ''
    if len(sequence) % 3 != 0:
        print("Warning: Sequence length is not a multiple of 3.")
    for i in range(0, len(sequence), 3):
        codon = sequence[i:i+3]
        if codon in codon_table:
            protein += codon_table[codon]
        else:
            protein += 'X'  # unknown amino acid
    return protein

In [ ]:
#Codon frequencies based on Human table from GenScript https://www.genscript.com/tools/codon-frequency-table

#These have been double checked for accuracy 
most_frequent_codon = {"F": "TTC", "L": "CTG", "Y": "TAC", "_":"TGA", "H":"CAC", "Q":"CAG", "I":"ATC", "M":"ATG", 
                      "N":"AAC", "K":"AAG", "V":"GTG", "D":"GAC", "E":"GAG", "S":"AGC", "C":"TGC", "W":"TGG",
                       "P":"CCC", "R":"CGG", "T": "ACC", "A":"GCC", "G":"GGC"}

#None means there is only one codon
second_most_frequent_codon = {"F": "TTT", "L": "CTC", "Y": "TAT", "_":"TAA", "H":"CAT", "Q":"CAA", "I":"ATT", "M":None, 
                      "N":"AAT", "K":"AAA", "V":"GTC", "D":"GAT", "E":"GAA", "S":"TCC", "C":"TGT", "W":None,
                       "P":"CCT", "R":"AGA", "T": "ACA", "A":"GCT", "G":"GGG"}

In [ ]:
def read_fasta_file(filename): #TESTED
    """
    Reads a FASTA file and returns the sequence as a string.

    Arguments:
    filename -- the name of the FASTA file to read

    Returns:
    A string representing the sequence in the FASTA file.
    """
    sequence = ""
    with open(filename, "r") as f:
        for line in f:
            if not line.startswith(">"):
                sequence += line.strip()
    return sequence.upper() #return the sequence in upper case to ensure compatibility with the downstream code (e.g. genetic table search)

In [ ]:
def reverse_complement(sequence): #TESTED
    """
    Returns the reverse complement of a nucleotide sequence.

    Arguments:
    sequence -- a string representing the nucleotide sequence

    Returns:
    The reverse complement of the input sequence, as a string.
    """
    complement = {"A": "T", "T": "A", "C": "G", "G": "C"}
    rev_comp = ""
    for base in reversed(sequence):
        if base not in complement:
            raise ValueError("Invalid nucleotide: {}".format(base))
        rev_comp += complement[base]
    return rev_comp


In [ ]:
cd Desktop

In [ ]:
def load_csv_to_list_of_dict(filename):
    """Loading the file that has been used as an input for PRIDICT as a list of dictionaries"""
    result = []
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        for i, row in enumerate(reader):
            row['id'] = i+1
            result.append(row)
    return result

def append_csv_data(data, folder_path, seq, old = 1, long = 0):
    """Function to append PRIDICT analysis to each designed pegRNA (we're takig the first row with matching spacer)
    seq is the full FASTA sequence of the region that is modified (including the flanking sites)
    We perform checks whether spacer from PRIDICT matches the intended SPACER
    
    We use the flag 'old' to make the code compatible with the old output of PRIDICT
    in the old output, the name of Spacer column was 'Spacer-Sequence'
    in the new output, the name of Spacer column is 'Protospacer-Sequence'
    
    We use the flag 'long' to specify the form of the PRIDICT output file
    In case we used the long PAM distance, long should be 1
    In case we used the short PAM distance, long should be 0"""
    for d in data:
        id_value = d['id']
        if(long == 0):
            filename = folder_path + '/' + str(id_value) + '_pegRNA_Pridict_full.csv'
        elif(long == 1):
            filename = folder_path + '/' + str(id_value) + '_long_pegRNA_Pridict_full.csv'

        if(d['Strand orientation'] == '+' or d['Strand orientation'] == '-'): #If it's not it means we couldn't find a nearby PAM
            try:
                expected_spacer = find_spacer(seq, d['Strand orientation'], int(d['Position of the first G in the PAM (sense direction)']))
                with open(filename, 'r') as f:
                    reader = csv.DictReader(f)
                    correct_spacer = 0
                    while(correct_spacer == 0): #Search for the first correct spacer (the data is sorted with respect to editing efficiency so we just need to look until the first correct spacer)
                        new_data = next(reader) #read the first next row into a dictionary  
                        if(old == 1):
                            pridict_spacer = new_data['Spacer-Sequence']

                        elif(old == 0):
                            pridict_spacer = new_data['Protospacer-Sequence']
                        if(pridict_spacer == expected_spacer):
                            correct_spacer = 1
                    d.update(new_data)
            except:
                #The csv was well output by PRIDICT but there were not cases where expected spacer matched the PRIDICT spacer
                print("Failed to find a matching spacer for " + str(id_value))
                d.update({'PRIDICT_editing_Score_deep': -1.0}) #we put the score -1 which means this entry should be neglected (otherwise there is an error when searching for highest score)

        else:
            print("Failed to open " + str(id_value))
            d.update({'PRIDICT_editing_Score_deep': -1.0}) #we put the score -1 which means this entry should be neglected (otherwise there is an error when searching for highest score)
    return data

def group_dict_by_keys(list_of_dicts, keep_only_frequent_codons = 0):
    """
    Takes a list of dictionaries and groups them by the first three keys.
    Returns a new list of dictionaries with the grouped dictionaries nested
    within their respective parent dictionaries.
    
    if keep_only_frequent_codons is 1, we keep only the otpimal codon for a mutation
    (the rest is not present in the output). For designing a synonymous change,
    in case the original codon is already optimal, we take the second most optimal.
    In case of W and M, we don't output anything (there are no possible mutations)
    """
    grouped_dict_list = []
    i = 0
    for d in list_of_dicts:
        values = list(d.values())
        key = (values[1], values[0], values[3]) #original aa, pos, mutated aa
        matching_dict = None
        
        for group in grouped_dict_list:
            group_key = (group['original_aa'], group['pos'], group['mutated_aa'])
            if group_key == key:
                matching_dict = group
                break
                
        if matching_dict is None:
            matching_dict = dict(zip(('original_aa', 'pos', 'mutated_aa'), key))
            grouped_dict_list.append(matching_dict)
            
        if(keep_only_frequent_codons == 0):
            matching_dict.setdefault('items', []).append(d)
        
        if(keep_only_frequent_codons == 1):
            if(key[0] != key[2]): #Non-synonynous change
                if(d['Mutated_nts'] == most_frequent_codon[d['Mutated_aa']]): #if the codon we're looking at is optimal
                    matching_dict.setdefault('items', []).append(d)         
                    
            elif(key[0] == key[2]): #Synonymous change
                if(d['Original_nts'] != most_frequent_codon[d['Original_aa']]): #if existing codon is not optimal
                    if(d['Mutated_nts'] == most_frequent_codon[d['Mutated_aa']]): #if we're looking at the optimal codon
                        matching_dict.setdefault('items', []).append(d)  
                        
                if(d['Original_nts'] == most_frequent_codon[d['Original_aa']]): #if existing codon is already optimal
                    if(d['Mutated_nts'] == second_most_frequent_codon[d['Mutated_aa']]): #if we're looking at the second best codon
                        matching_dict.setdefault('items', []).append(d)
        if(i%1000 == 0):
            print(i)
        i += 1
    return grouped_dict_list  

def merge_grouped_dicts(list1, list2): #TESTED
    """Merges two grouped dictionaries
    Should be used before finding the best score
    We do not perform checks to see if we're adding duplicates - in any case only one will be chosen at the end by max score"""
    merged_list = []
    
    for dict1 in list1:
        merged_dict = dict1.copy()
        
        for dict2 in list2:
            if dict1['original_aa'] == dict2['original_aa'] and dict1['pos'] == dict2['pos'] and dict1['mutated_aa'] == dict2['mutated_aa']:
                merged_dict['items'] = dict1.get('items', []) + dict2.get('items', [])
        merged_list.append(merged_dict)
    
    for dict2 in list2:
        is_unique = True
        
        for dict1 in list1:
            if dict1['original_aa'] == dict2['original_aa'] and dict1['pos'] == dict2['pos'] and dict1['mutated_aa'] == dict2['mutated_aa']:
                is_unique = False
                break
        
        if is_unique:
            merged_list.append(dict2)
    
    return merged_list

    
def find_highest_score(scores): #TESTED
    """Finds the dictionary with the highest 'PRIDICT_editing_Score_deep' value' among all dictionaries with the same (original_aa, position, mutated_aa)"""
    highest_score = None
    
    for score in scores:
        if highest_score is None or float(score['PRIDICT_editing_Score_deep']) > float(highest_score['PRIDICT_editing_Score_deep']):
            highest_score = score
            
    return highest_score

def print_highest_score_values(data, output_file): #TESTED
    # Extract the header keys from the first nested dictionary
    items_keys = list(data[0]['items'][0].keys())

    # Open the output file and create a CSV writer
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)

        # Write the header row
        writer.writerow(items_keys)

        # Loop over the dictionaries in the list
        for i,d in enumerate(data):
            # Find the dictionary with the highest 'score' value
            max_score_dict = find_highest_score(d['items'])

            # Write its values to the CSV file
            if(max_score_dict != None):
                writer.writerow(max_score_dict.values())


In [ ]:
def find_spacer(string, orientation, pam_pos): #TESTED
    """This function finds spacer based on PAM position
    It changes the first nucleotide of the spacer into G, same as PRIDICT"""
    
    pam_pos = pam_pos-1 #We load from excel where indexing is 1-based to Python where it's 0-based
    if orientation == '+':
        start = pam_pos - 21
        end = pam_pos - 1
        substring = string[start:end]
    elif orientation == '-':
        start = pam_pos + 3
        end = pam_pos + 23
        substring = string[start:end]
        substring = reverse_complement(substring)
    else:
        raise ValueError("Invalid orientation parameter: must be '+' or '-'")

    return 'G' + substring[1:]  # Change first character to 'G' (this is already performed by PRIDICT output)

In [ ]:
def reconstruct_whole_mut_seq(seq, original_subseq, mutated_subseq, strand_orientation):
    """Finds whole mutate sequence (same length as input fasta) 
    seq: original input fasta
    original_subseq: original seq fed to predict
    mutated_seq: mutated_seq fed to pridict
    strand orientation is needed because in case we use Rv strand, PRIDICT will use reverse complement for original_subseq
    and mutated_subseq while seq will be read from Fw strand"""
    
    #In newer versions of PRIDICT, the output contains small letters in the edited region
    #This is not compatible with the rest of the pipeline so we have convert it to upper
    mutated_subseq = mutated_subseq.upper() 
    
    if(strand_orientation == '-'):
        original_subseq = reverse_complement(original_subseq)
        mutated_subseq = reverse_complement(mutated_subseq)

    index = seq.find(original_subseq)
    L = len(mutated_subseq)
    
    seq1 = seq[:index]
    seq1 += mutated_subseq
    seq1 += seq[index+L:]
    
    return seq1

In [ ]:
def mutated_seed(original_seq, mutated_seq, pam_pos, orientation, printing = 0): #TESTED
    """Verifies that the seed is mutated"""
    pam_pos = pam_pos - 1 #We load from excel where indexing is 1-based to Python where it's 0-based
    #Inputs should have the same lengths as the input FASTA so that pam_pos is correct
    #Note thatt pam_pos is the position of the first G in the case of + orientation
    #Or the position of first C in the + strand in the caseof - orientation
    if(orientation == "+"):
        seed_pos = pam_pos - 4
    if(orientation == "-"):
        seed_pos = pam_pos + 3
    
    seed_original = original_seq[seed_pos:seed_pos + 3]

    seed_mutated = mutated_seq[seed_pos:seed_pos+3]
    
    if(printing == 1):
        print('seed original')
        print(seed_original)
        print('seed mutated')
        print(seed_mutated)
    if(seed_original != seed_mutated):
        return 1
    else:
        return 0

In [ ]:
def find_mutated_codon(mutated_seq, aa_pos):
    return translate(mutated_seq[aa_pos:aa_pos+3])

In [ ]:
def remove_bad_pegRNAs(grouped_dict_list, seq):
    """Removes the dictionaries that are considered bad by fullfilling one of the following criteria:
    1. The expected spacer seq does not match the spacer sequence assumed by PRIDICT
    2. The seed is not actually mutated
    3. (add later) There are unnecessary mutations in the PAM-containing AAs that do not disrupt PAM"""
    new_grouped_dict_list = []
    i = 0
    for d in grouped_dict_list:
        #setting the header of the dict
        new_dict = {}
        new_dict['original_aa'] = d['original_aa']
        new_dict['pos'] = d['pos']
        new_dict['mutated_aa'] = d['mutated_aa']
        new_dict['items'] = []
        
        #if(d['original_aa']=='C' and d['pos'] == '270' and d['mutated_aa'] == 'K'): #Was used for testing
        #if(d['original_aa']=='S' and d['pos'] == '219' and d['mutated_aa'] == 'S'):
        
        #iterating over items to filter out bad designs
        for item in d['items']:
            if(item['PRIDICT_editing_Score_deep'] != -1): #This happens in some cases, we need to avoid them so that it runs through
                if(int(item['Total Hamming distance']) > 1):
                    expected_mut_aa = item['Mutated_aa']
                    whole_mut_seq = reconstruct_whole_mut_seq(seq, item['Original_Sequence'], item['Edited-Sequences'], item['Strand orientation'])
                    aa_pos = int(item['Position of the first nt in the aa'])-1 #Converting to Python 0-based indexing
                    actual_mut_aa = find_mutated_codon(whole_mut_seq, aa_pos)

                    if(expected_mut_aa == actual_mut_aa):
                        expected_spacer = find_spacer(seq, item['Strand orientation'], int(item['Position of the first G in the PAM (sense direction)']))
                        pridict_spacer = expected_spacer #item['Spacer-Sequence']
                        #print('=====')
                        #print(item['Position of the first G in the PAM (sense direction)'])
                        #print(expected_spacer)
                        #print(pridict_spacer)

                        #condition 1
                        if(expected_spacer == pridict_spacer):
                            #condition 2 if seed is expected to be mutated
                            if(item['Seed mutated'] == '1' and mutated_seed(seq, whole_mut_seq, int(item['Position of the first G in the PAM (sense direction)']), item['Strand orientation'])):
                                #Keep only these
                                new_dict['items'].append(item)  
                            elif(item['Seed mutated'] == '0'): #if seed is not expected to be mutated, we just print out
                                new_dict['items'].append(item)
                    else:
                        print('Hamming > 1 but AA mut not well designed')
                        print(item['id'])
                        print(item['Mutated_nts'])
                        print(seq[aa_pos-6:aa_pos+9])
                        print(seq[aa_pos:aa_pos+3])
                        print('Reconstructed mut seq')
                        print(whole_mut_seq[aa_pos-6:aa_pos+9])
                        print(whole_mut_seq[aa_pos:aa_pos+3])
                        print(translate(whole_mut_seq[aa_pos:aa_pos+3]))

        new_grouped_dict_list.append(new_dict)
        i = i+1
    return new_grouped_dict_list

In [ ]:
data = load_csv_to_list_of_dict('pegRNA_main_initial_design.csv')
seq = read_fasta_file('LDLR ex4_extra seq.fa') #Reading the fasta file (has to be in .fa format and in the same folder as the code)
data = append_csv_data(data, 'predictions main', seq, old = 1, long = 0)
grouped_dict_list = group_dict_by_keys(data, keep_only_frequent_codons = 1)

In [ ]:
data_additional_seed = load_csv_to_list_of_dict('pegRNA_design_additional_seed_mutations.csv')
data_additional_seed = append_csv_data(data_additional_seed, 'predictions additional seed', seq, old = 0, long = 0)
grouped_dict_list2 = group_dict_by_keys(data_additional_seed, keep_only_frequent_codons = 1)

In [ ]:
data_two_more_aa = load_csv_to_list_of_dict('pegRNA_design_324_and_327_with_35_limit_mutations.csv')
data_two_more_aa  = append_csv_data(data_two_more_aa, 'predictions longer limit', seq, old = 0, long = 1)
grouped_dict_list3 = group_dict_by_keys(data_two_more_aa, keep_only_frequent_codons = 1)

In [ ]:
data_three_more_aa = load_csv_to_list_of_dict('pegRNA_design_264_417_420.csv')
data_three_more_aa  = append_csv_data(data_two_more_aa, 'predictions long three more', seq, old = 0, long = 0)
grouped_dict_list4 = group_dict_by_keys(data_two_more_aa, keep_only_frequent_codons = 1)

In [ ]:
merged_dicts_tmp = merge_grouped_dicts(grouped_dict_list, grouped_dict_list2)
merged_dicts_tmp2 = merge_grouped_dicts(merged_dicts_tmp, grouped_dict_list3)
merged_dicts = merge_grouped_dicts(merged_dicts_tmp2, grouped_dict_list4)

In [ ]:
grouped_dict_list_new = remove_bad_pegRNAs(merged_dicts, read_fasta_file('LDLR ex4_extra seq.fa'))

In [ ]:
#Printing the final output
print_highest_score_values(grouped_dict_list_new, 'Filtered_pegRNAs_with_additional_seed_and_longer_limit_for_five_aas.csv')

In [ ]:
#Identifying potentially missing mutations
def find_missing_mutations(csv_file):
    """Goes through the final output of the pipeline and lists all amino-acids for which a mutation could not be designed"""
    positions = []
    mutations = []

    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row

        for row in reader:
            position = int(row[0])
            mutation = row[3]

            positions.append(position)
            mutations.append(mutation)

    unique_positions = set(positions)
    missing_mutations = {}

    for position in unique_positions:
        expected_mutations = set(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '_'])
        observed_mutations = set([mutation for idx, mutation in enumerate(mutations) if positions[idx] == position])

        missing_mutations[position] = list(expected_mutations - observed_mutations)

    return missing_mutations

missing_mutations = find_missing_mutations('Filtered_pegRNAs_with_additional_seed_and_longer_limit_for_five_aas.csv')

In [ ]:
missing_mutations